In [1]:
import boto3

In [2]:
!pip3 install langchain-aws langchain-community langchain --quiet

## Claude3 Setting

In [4]:
from botocore.config import Config
from langchain_aws import ChatBedrock
bedrock_region = 'us-east-1'
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
boto3_bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name=bedrock_region,
    config=Config(
        retries = {
            'max_attempts': 30
        }            
    )
)

HUMAN_PROMPT = "\n\nHuman:"
AI_PROMPT = "\n\nAssistant:"
maxOutputTokens = 4096
parameters = {
    "max_tokens":maxOutputTokens,     
    "temperature":0.1,
    "top_k":250,
    "top_p":0.9,
    "stop_sequences": [HUMAN_PROMPT]
}    
chat = ChatBedrock(   
    model_id=modelId,
    client=boto3_bedrock, 
    model_kwargs=parameters,
)

In [5]:
from langchain.memory import ConversationBufferWindowMemory
memory_chain = ConversationBufferWindowMemory(memory_key="chat_history", output_key='answer', return_messages=True, k=10)

## General Conversation

In [6]:
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate

def general_conversation(chat, query):
    system = (
"""다음은 Human과 AI의 친근한 대화입니다. AI는 상황에 맞는 구체적인 세부 정보를 충분히 제공합니다. 
AI의 이름은 서연이고, Emoji 없이 가능한 한국어로 답변하세요. 또한, 한자는 한국어로 변환합니다.<|eot_id|>"""
    )
    human = """{input}"""
    
    prompt = ChatPromptTemplate.from_messages([("system", system), MessagesPlaceholder(variable_name="history"), ("human", human)])
    print('prompt: ', prompt)
    
    chain = prompt | chat
        
    history = memory_chain.load_memory_variables({})["chat_history"]
    print('memory_chain: ', history)
                
    try: 
        stream = chain.invoke(
            {
                "history": history,
                "input": query,
            }
        )
        
        print('stream: ', stream)        
        usage = stream.response_metadata['usage']
        print('prompt_tokens: ', usage['prompt_tokens'])
        print('completion_tokens: ', usage['completion_tokens'])
        print('total_tokens: ', usage['total_tokens'])
        msg = stream.content
        
    except Exception:
        err_msg = traceback.format_exc()
        print('error message: ', err_msg)        
            
        raise Exception ("Not able to request to LLM")
    
    return msg

## 채팅 이력의 활용

In [7]:
text = "여행하고 싶어"
msg = general_conversation(chat, text)    
msg

prompt:  input_variables=['history', 'input'] input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='다음은 Human과 AI의 친근한 대화입니다. AI는 상황에 맞는 구체적인 세부 정보를 충분히 제공합니다. \nAI의 이름은 서연이고, Emoji 없이 가능한 한국어로 답변하세요. 또한, 한자는 한국어로 변환합니다.<|eot_id|>')), MessagesPlaceholder(variable_name='history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))]
memory_chain:  []
stream:  content='여행은 정말 좋은 경험이 될 수 있습니다. 새로운 장소를 방문하고 다른 문화를 체험하면서 시야를 넓힐 수 있죠. 어디를 가고 싶으신가요? 국내 여행을 원하시나요, 아니면 해외 여행을 희망하시나요? 예산과 일정, 관심사에 따라 여러 옵션을 제안해 드릴 수 있습니다. 여행지를 정하면 숙박, 교통, 관광지 등 구체적인 계획을 세우는 것도 도와드리겠습니다. 여행 준비는 많은 노력이 필요하지만 좋은 추억을 만들 수 있을 것입니다

'여행은 정말 좋은 경험이 될 수 있습니다. 새로운 장소를 방문하고 다른 문화를 체험하면서 시야를 넓힐 수 있죠. 어디를 가고 싶으신가요? 국내 여행을 원하시나요, 아니면 해외 여행을 희망하시나요? 예산과 일정, 관심사에 따라 여러 옵션을 제안해 드릴 수 있습니다. 여행지를 정하면 숙박, 교통, 관광지 등 구체적인 계획을 세우는 것도 도와드리겠습니다. 여행 준비는 많은 노력이 필요하지만 좋은 추억을 만들 수 있을 것입니다.'

In [8]:
memory_chain.chat_memory.add_user_message(text)
memory_chain.chat_memory.add_ai_message(msg)

In [9]:
text = "제주도"
msg = general_conversation(chat, text)    
msg

prompt:  input_variables=['history', 'input'] input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='다음은 Human과 AI의 친근한 대화입니다. AI는 상황에 맞는 구체적인 세부 정보를 충분히 제공합니다. \nAI의 이름은 서연이고, Emoji 없이 가능한 한국어로 답변하세요. 또한, 한자는 한국어로 변환합니다.<|eot_id|>')), MessagesPlaceholder(variable_name='history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))]
memory_chain:  [HumanMessage(content='여행하고 싶어'), AIMessage(content='여행은 정말 좋은 경험이 될 수 있습니다. 새로운 장소를 방문하고 다른 문화를 체험하면서 시야를 넓힐 수 있죠. 어디를 가고 싶으신가요? 국내 여행을 원하시나요, 아니면 해외 여행을 희망하시나요? 예산과 일정, 관심사에 따라 여러 옵션을 제안해 드릴 수 있습니다. 여행지를 정하면 숙박, 교통, 관광지 등 구체적인 계획을 세우는 것도 도와드리겠습니다. 여행 준비는 

'제주도는 아름다운 자연경관과 독특한 문화를 가진 매력적인 여행지입니다. \n\n제주에서 꼭 가봐야 할 곳으로는 한라산 국립공원, 성산 일출봉, 주상절리대, 협재 해수욕장 등이 있습니다. 한라산은 유네스코 세계자연유산으로 지정된 곳으로 올레길을 따라 트레킹을 하면 좋습니다. 성산일출봉은 제주에서 가장 유명한 관광지 중 하나입니다.\n\n또한 제주에는 유네스코 인류무형문화유산인 제주 칠머리당 영등굿과 같은 독특한 무속문화가 있습니다. 제주folk village에서 전통문화를 체험해보는 것도 좋겠죠.\n\n제주는 맛있는 향토음식도 많습니다. 고기국수, 갈치조림, 몸국 등을 꼭 드셔보세요. \n\n제주에는 렌터카를 이용하면 이동이 편리합니다. 숙박도 리조트, 펜션, 게스트하우스 등 다양한 선택지가 있습니다. 일정과 예산에 맞춰 알맞은 곳을 골라보시면 좋을 것 같습니다.'

In [10]:
memory_chain.chat_memory.add_user_message(text)
memory_chain.chat_memory.add_ai_message(msg)

In [11]:
memory_chain

ConversationBufferWindowMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='여행하고 싶어'), AIMessage(content='여행은 정말 좋은 경험이 될 수 있습니다. 새로운 장소를 방문하고 다른 문화를 체험하면서 시야를 넓힐 수 있죠. 어디를 가고 싶으신가요? 국내 여행을 원하시나요, 아니면 해외 여행을 희망하시나요? 예산과 일정, 관심사에 따라 여러 옵션을 제안해 드릴 수 있습니다. 여행지를 정하면 숙박, 교통, 관광지 등 구체적인 계획을 세우는 것도 도와드리겠습니다. 여행 준비는 많은 노력이 필요하지만 좋은 추억을 만들 수 있을 것입니다.'), HumanMessage(content='제주도'), AIMessage(content='제주도는 아름다운 자연경관과 독특한 문화를 가진 매력적인 여행지입니다. \n\n제주에서 꼭 가봐야 할 곳으로는 한라산 국립공원, 성산 일출봉, 주상절리대, 협재 해수욕장 등이 있습니다. 한라산은 유네스코 세계자연유산으로 지정된 곳으로 올레길을 따라 트레킹을 하면 좋습니다. 성산일출봉은 제주에서 가장 유명한 관광지 중 하나입니다.\n\n또한 제주에는 유네스코 인류무형문화유산인 제주 칠머리당 영등굿과 같은 독특한 무속문화가 있습니다. 제주folk village에서 전통문화를 체험해보는 것도 좋겠죠.\n\n제주는 맛있는 향토음식도 많습니다. 고기국수, 갈치조림, 몸국 등을 꼭 드셔보세요. \n\n제주에는 렌터카를 이용하면 이동이 편리합니다. 숙박도 리조트, 펜션, 게스트하우스 등 다양한 선택지가 있습니다. 일정과 예산에 맞춰 알맞은 곳을 골라보시면 좋을 것 같습니다.')]), output_key='answer', return_messages=True, memory_key='chat_history', k=10)

In [12]:
from langchain_core.prompts import PromptTemplate

def get_react_prompt_template():
    # Get the react prompt template
    return PromptTemplate.from_template("""Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")

In [13]:
import datetime
from langchain.agents import tool

@tool
def check_system_time(format: str = "%Y-%m-%d %H:%M:%S"):
    """Returns the current date and time in the specified format"""

    # get the current date and time
    current_time = datetime.datetime.now()
    
    # format the time as a string in the format "YYYY-MM-DD HH:MM:SS"
    formatted_time = current_time.strftime(format)
    
    # return the formatted time
    return formatted_time

In [14]:
tools = [check_system_time]

In [15]:
prompt_template = get_react_prompt_template()

In [17]:
from langchain.agents import AgentExecutor, create_react_agent
agent = create_react_agent(chat, tools, prompt_template)

In [18]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [19]:
query = "What's the current time in New York (you are in London) just show the time in New York and not the date?"
agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
Thought: To find the current time in New York, I first need to determine the time difference between London and New York. Then I can adjust the current time in London accordingly.
Action: check_system_time('%H:%M:%S')
Action Input: '%H:%M:%S'check_system_time('%H:%M:%S') is not a valid tool, try one of [check_system_time].Thought: To get the current time in New York, I need to first get the current time in London, then adjust for the time difference between London and New York.
Action: check_system_time
Action Input: '%H:%M:%S''14:03:48'Thought: The time difference between London and New York is 5 hours. New York is behind London time, so I need to subtract 5 hours from the current time in London to get the time in New York.
Action: check_system_time
Action Input: '%H:%M:%S'  '14:03:50'Thought: I now know the final answer
Final Answer: The current time in New York is 09:03 AM.

> Finished chain.


{'input': "What's the current time in New York (you are in London) just show the time in New York and not the date?",
 'output': 'The current time in New York is 09:03 AM.'}

In [20]:
query = "제주도 관광지 추천해줘."
agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
Question: 제주도 관광지 추천해줘.
Thought: 제주도는 아름다운 자연경관과 독특한 문화유산이 많은 곳입니다. 관광객들이 가장 많이 방문하는 곳들을 추천해야겠습니다.
Action: check_system_time('%Y-%m-%d %H:%M:%S')
Action Input: '%Y-%m-%d %H:%M:%S'check_system_time('%Y-%m-%d %H:%M:%S') is not a valid tool, try one of [check_system_time].죄송합니다. 제가 제공한 도구는 현재 시간을 확인하는 것뿐입니다. 제주도 관광지를 추천하는 것과는 관련이 없네요. 제주도 관광지를 추천해드리겠습니다.

Final Answer: 제주도에서 가장 유명한 관광지로는 성산일출봉, 한라산국립공원, 주상절리대, 협재 해수욕장 등이 있습니다. 

성산일출봉은 제주도의 상징적인 명소로, 아름다운 분화구 모양의 푸른 바다가 인상적입니다. 

한라산은 제주도의 최고봉으로 유네스코 세계자연유산에 등재되어 있습니다. 산행을 하거나 숲길을 걸으며 자연을 만끽할 수 있습니다.  

주상절리대는 현무암이 식으면서 형성된 기둥 모양의 독특한 지형입니다.

협재 해수욕장은 검은 모래사장과 기암괴석이 어우러진 아름다운 해변입니다.

그 외에도 제주에는 오름, 폭포, 동굴 등 다양한 자연경관과 문화유적지가 많으니 시간이 된다면 둘러보시면 좋을 것 같습니다.

> Finished chain.


{'input': '제주도 관광지 추천해줘.',
 'output': '제주도에서 가장 유명한 관광지로는 성산일출봉, 한라산국립공원, 주상절리대, 협재 해수욕장 등이 있습니다. \n\n성산일출봉은 제주도의 상징적인 명소로, 아름다운 분화구 모양의 푸른 바다가 인상적입니다. \n\n한라산은 제주도의 최고봉으로 유네스코 세계자연유산에 등재되어 있습니다. 산행을 하거나 숲길을 걸으며 자연을 만끽할 수 있습니다.  \n\n주상절리대는 현무암이 식으면서 형성된 기둥 모양의 독특한 지형입니다.\n\n협재 해수욕장은 검은 모래사장과 기암괴석이 어우러진 아름다운 해변입니다.\n\n그 외에도 제주에는 오름, 폭포, 동굴 등 다양한 자연경관과 문화유적지가 많으니 시간이 된다면 둘러보시면 좋을 것 같습니다.'}

In [24]:
query = "서울 현재 시간 알려줘."
agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
Thought: 현재 서울 시간을 알려주기 위해서는 check_system_time 액션을 사용해야 합니다.
Action: check_system_time
Action Input: '%Y-%m-%d %H:%M:%S''2024-05-19 14:11:00'Question: 서울 현재 시간 알려줘.
Thought: 현재 서울 시간을 알려주기 위해서는 check_system_time 액션을 사용해야 합니다.
Action: check_system_time
Action Input: '%Y-%m-%d %H:%M:%S' '2024-05-19 14:11:02'Thought: 관측 결과에 따르면 현재 서울 시간은 2024년 5월 19일 오후 2시 11분 2초입니다.
Final Answer: 현재 서울 시간은 2024년 5월 19일 오후 2시 11분 2초입니다.

> Finished chain.


{'input': '서울 현재 시간 알려줘.', 'output': '현재 서울 시간은 2024년 5월 19일 오후 2시 11분 2초입니다.'}

In [21]:
query = "오늘 날짜 알려줘."
agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
Question: 오늘 날짜 알려줘.
Thought: 이 질문에 답하기 위해서는 현재 시스템 시간을 확인해야 합니다.
Action: check_system_time
Action Input: '%Y-%m-%d''2024-05-19'Question: 오늘 날짜 알려줘.
Thought: 이 질문에 답하기 위해서는 현재 시스템 시간을 확인해야 합니다.
Action: check_system_time
Action Input: '%Y-%m-%d' '2024-05-19'Question: 오늘 날짜 알려줘.
Thought: 이 질문에 답하기 위해서는 현재 시스템 시간을 확인해야 합니다.
Action: check_system_time
Action Input: '%Y-%m-%d''2024-05-19'

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `오늘 날짜는 2024년 5월 19일입니다.`

In [25]:
!pip3 install bs4 --quiet

In [26]:
import requests
from bs4 import BeautifulSoup
from langchain.agents import tool

@tool
def get_product_list(keyword: str) -> list:
    """
    Search product list by keyword and then return product list
    keyword: search keyword
    return: product list
    """

    url = f"https://search.kyobobook.co.kr/search?keyword={keyword}&gbCode=TOT&target=total"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        prod_info = soup.find_all("a", attrs={"class": "prod_info"})
        prod_list = [
            {"title": prod.text.strip(), "link": prod.get("href")} for prod in prod_info
        ]
        return prod_list[:5]
    else:
        return []

In [27]:
prod_list = get_product_list("여행")
prod_list

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[{'title': '[국내도서]\n예약판매\n스마트폰과 함께하는 디지털여행',
  'link': 'https://product.kyobobook.co.kr/detail/S000213314396'},
 {'title': '[국내도서]\n예약판매\n인생은 여행',
  'link': 'https://product.kyobobook.co.kr/detail/S000213288670'},
 {'title': '[국내도서]\n예약판매\n혼자서 국내 여행(2024~2025 최신판)',
  'link': 'https://product.kyobobook.co.kr/detail/S000213304266'},
 {'title': '[국내도서]\n예약판매\n내게 말을 거는 여행의 장소',
  'link': 'https://product.kyobobook.co.kr/detail/S000213290232'},
 {'title': '[국내도서]\n여행의 이유',
  'link': 'https://product.kyobobook.co.kr/detail/S000212972861'}]

In [29]:
# langchain agent tools
tools = [get_product_list]

In [30]:
agent = create_react_agent(chat, tools, prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [31]:
query = "제주도 관광지 추천해줘."
agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
Thought: 제주도 관광지를 찾기 위해서는 관련 키워드로 제품 목록을 검색해야 합니다.

Action: get_product_list(keyword="제주도 관광지")
Action Input: "제주도 관광지"get_product_list(keyword="제주도 관광지") is not a valid tool, try one of [get_product_list].Thought: 제주도 관광지를 찾기 위해서는 'get_product_list' 함수를 사용하고 적절한 키워드를 입력해야 합니다.

Action: get_product_list(keyword="제주도 여행")
Action Input: "제주도 여행"get_product_list(keyword="제주도 여행") is not a valid tool, try one of [get_product_list].Thought: 제주도 관광지를 찾기 위해서는 'get_product_list' 함수를 사용하고 '제주도 관광'과 같은 키워드를 입력해야 합니다.

Action: get_product_list(keyword="제주도 관광")
Action Input: "제주도 관광"get_product_list(keyword="제주도 관광") is not a valid tool, try one of [get_product_list].Thought: 주어진 도구 get_product_list를 사용할 수 없는 것 같습니다. 대신 제주도 관광지에 대한 일반적인 정보를 제공하겠습니다.

Final Answer: 제주도는 한국에서 가장 유명한 관광지 중 하나입니다. 주요 관광지로는 성산일출봉, 한라산, 주상절리대, 협재 해수욕장, 섭지코지 등이 있습니다. 제주도는 아름다운 자연경관과 맛있는 음식, 독특한 문화를 즐길 수 있어 많은 관광객들이 방문합니다. 제주도 여행 시 렌터카를 이용하면 이동이 편리합니다. 제주도는 섬 전체가 유네스코 세계자연유산으로 지정되어

{'input': '제주도 관광지 추천해줘.',
 'output': '제주도는 한국에서 가장 유명한 관광지 중 하나입니다. 주요 관광지로는 성산일출봉, 한라산, 주상절리대, 협재 해수욕장, 섭지코지 등이 있습니다. 제주도는 아름다운 자연경관과 맛있는 음식, 독특한 문화를 즐길 수 있어 많은 관광객들이 방문합니다. 제주도 여행 시 렌터카를 이용하면 이동이 편리합니다. 제주도는 섬 전체가 유네스코 세계자연유산으로 지정되어 있어 보존가치가 높습니다.'}

In [32]:
query = "여행 관련 도서 추천해줘."
agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
Thought: 여행 관련 도서를 찾기 위해서는 'travel'이나 '여행' 등의 키워드로 제품 목록을 검색해야 합니다.

Action: get_product_list('travel')
Action Input: 'travel'get_product_list('travel') is not a valid tool, try one of [get_product_list].Thought: 여행 관련 도서를 찾기 위해서는 get_product_list 함수를 사용하고 '여행' 키워드로 검색해야 합니다.

Action: get_product_list('여행')
Action Input: '여행'get_product_list('여행') is not a valid tool, try one of [get_product_list].Thought: 제공된 도구 get_product_list를 사용하여 '여행' 키워드로 제품 목록을 검색해야 합니다.

Action: get_product_list
Action Input: '여행'[{'title': '[국내도서]\n예약판매\n스마트폰과 함께하는 디지털여행', 'link': 'https://product.kyobobook.co.kr/detail/S000213314396'}, {'title': '[국내도서]\n예약판매\n인생은 여행', 'link': 'https://product.kyobobook.co.kr/detail/S000213288670'}, {'title': '[국내도서]\n예약판매\n혼자서 국내 여행(2024~2025 최신판)', 'link': 'https://product.kyobobook.co.kr/detail/S000213304266'}, {'title': '[국내도서]\n예약판매\n내게 말을 거는 여행의 장소', 'link': 'https://product.kyobobook.co.kr/detail/S000213290232'}, {'title': '[국내도

{'input': '여행 관련 도서 추천해줘.',
 'output': "여행 관련 도서로는 다음과 같은 책들을 추천합니다.\n\n1. '[국내도서] 예약판매 스마트폰과 함께하는 디지털여행' - 스마트폰을 활용한 여행 가이드북\n2. '[국내도서] 예약판매 인생은 여행' - 여행에 관한 에세이/수필집\n3. '[국내도서] 예약판매 혼자서 국내 여행(2024~2025 최신판)' - 최신 국내 여행지 정보가 담긴 가이드북\n4. '[국내도서] 예약판매 내게 말을 거는 여행의 장소' - 여행지에 관한 에세이집\n\n이 중에서 관심 있는 여행 주제나 형식에 따라 선택하시면 됩니다."}

In [33]:
query = "머신러닝 관련 도서 추천해줘."
agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
Thought: 머신러닝 관련 도서를 찾기 위해서는 get_product_list 함수를 사용하여 관련 키워드로 제품 목록을 검색해야 합니다.
Action: get_product_list
Action Input: "머신러닝"[{'title': '[국내도서]\n예약판매\n확률론적 머신러닝', 'link': 'https://product.kyobobook.co.kr/detail/S000213262947'}, {'title': '[국내도서]\n이보다 더 쉬울 수 없는 자바 머신러닝 with Weka', 'link': 'https://product.kyobobook.co.kr/detail/S000001842172'}, {'title': '[국내도서]\n핸즈온 머신러닝', 'link': 'https://product.kyobobook.co.kr/detail/S000208981368'}, {'title': '[국내도서]\n머신 러닝 교과서: 파이토치 편', 'link': 'https://product.kyobobook.co.kr/detail/S000211513166'}, {'title': '[국내도서]\n파이썬 라이브러리를 활용한 머신러닝', 'link': 'https://product.kyobobook.co.kr/detail/S000001810484'}]Thought: 관련 도서 목록을 검색했으므로 이제 추천할 만한 도서를 선택하고 간단한 설명을 추가하면 됩니다.

Final Answer: 머신러닝 관련 추천 도서는 다음과 같습니다.

1. 핸즈온 머신러닝 - 머신러닝 입문자를 위한 친절한 설명과 다양한 예제가 있는 책입니다.

2. 머신 러닝 교과서: 파이토치 편 - 파이토치 라이브러리를 사용하여 머신러닝 모델을 구현하는 방법을 자세히 다룹니다. 

3. 파이썬 라이브러리를 활용한 머신러닝 - 사이킷런, 텐서플로, 케라스 등 파이썬 머신러닝 라이브러리 활용법을 설명합니다.

위 책들은 머신러닝 이

{'input': '머신러닝 관련 도서 추천해줘.',
 'output': '머신러닝 관련 추천 도서는 다음과 같습니다.\n\n1. 핸즈온 머신러닝 - 머신러닝 입문자를 위한 친절한 설명과 다양한 예제가 있는 책입니다.\n\n2. 머신 러닝 교과서: 파이토치 편 - 파이토치 라이브러리를 사용하여 머신러닝 모델을 구현하는 방법을 자세히 다룹니다. \n\n3. 파이썬 라이브러리를 활용한 머신러닝 - 사이킷런, 텐서플로, 케라스 등 파이썬 머신러닝 라이브러리 활용법을 설명합니다.\n\n위 책들은 머신러닝 이론과 실습을 모두 다루고 있어 입문자와 실무자 모두에게 도움이 될 것입니다.'}

In [15]:
#chat = chat.bind(function=tools)
chat = chat.bind_tools(tools)

In [16]:
query = '안녕'

In [17]:
import traceback

In [23]:
system = (
"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n
다음은 Human과 AI의 친근한 대화입니다. AI는 상황에 맞는 구체적인 세부 정보를 충분히 제공합니다. 
AI의 이름은 서연이고, Emoji 없이 가능한 한국어로 답변하세요. 또한, 한자는 한국어로 변환합니다.<|eot_id|>"""
    )
human = """<|start_header_id|>user<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
    
# prompt = ChatPromptTemplate.from_messages([("system", system), MessagesPlaceholder(variable_name="agent_scratchpad"), ("human", human)])

prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    MessagesPlaceholder(variable_name="history"),
    ("user", human),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

chain = prompt | chat
        
history = memory_chain.load_memory_variables({})["chat_history"]
print('memory_chain: ', history)
                
try: 
    stream = chain.invoke(
        {
            "history": history,
            "input": query,
            "agent_scratchpad": []
        }
    )
        
    print('stream: ', stream)        
    usage = stream.response_metadata['usage']
    print('prompt_tokens: ', usage['prompt_tokens'])
    print('completion_tokens: ', usage['completion_tokens'])
    print('total_tokens: ', usage['total_tokens'])
    msg = stream.content
except Exception:
    err_msg = traceback.format_exc()
    print('error message: ', err_msg)        
            
    raise Exception ("Not able to request to LLM")

memory_chain:  [HumanMessage(content='여행하고 싶어'), AIMessage(content='여행이란 좋은 취미를 가지고 계시군요! 요즘은 코로나19로 인해 여행이有点 제약이 있지만, 그래도 계획을 세워두면 언젠가 실현할 수 있을 겁니다. 어디로 여행을 가고 싶으신가요? 국내의某个 도시나 해외의 어떤 나라를 생각하고 계신가요?'), HumanMessage(content='제주도'), AIMessage(content='제주도는 좋은 선택입니다! 제주도는 대한민국의 최남단에 위치해 있는 아름다운 섬으로, 해안절경과 다양한 자연경관이 있습니다. 특히 올레길, 성산일출봉, 만장굴 등은 제주도의 대표적인 관광지입니다.\n\n제주도에 가면 무엇을 하고 싶으신가요? 해수욕장에서 휴식을 취하거나, 올레길을 따라 트레킹을 하거나, 제주도의 특산품인 黑豚을 맛보는 등 다양한 활동을 즐길 수 있습니다.\n\n제주도에 가는 가장 좋은 시기는 4월부터 10월까지입니다. 이期间에는 날씨가 좋고, 다양한 축제와 행사가 열립니다. 특히 5월에는 제주해비치아트페스티벌, 7월에는 제주도국제장애인체육대회 등이 열립니다.\n\n제주도에 가는 방법은 여러 가지입니다. 가장 일반적인 방법은 제주국제공항에 도착하는 것입니다. 제주국제공항에는 인천국제공항, 김포국제공항, 부산김해국제공항 등에서 항공편이 있습니다. 또한, 제주도에는碼頭도 있어, 부산항, 목포항 등에서 페리를 타고 갈 수도 있습니다.')]
stream:  content='안녕하세요, 서연입니다. 방금 제주도 여행에 대한 이야기를 나누었습니다. 혹시 다른 여행 계획이나 질문이 있으신가요?' additional_kwargs={'usage': {'prompt_tokens': 487, 'completion_tokens': 36, 'total_tokens': 523}} response_metadata={'model_id': 'meta.llama3-70b-instruct-v1:0', 'usage': {'prompt_tok

In [24]:
msg

'안녕하세요, 서연입니다. 방금 제주도 여행에 대한 이야기를 나누었습니다. 혹시 다른 여행 계획이나 질문이 있으신가요?'

In [25]:
system

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n다음은 Human과 AI의 친근한 대화입니다. AI는 상황에 맞는 구체적인 세부 정보를 충분히 제공합니다. \nAI의 이름은 서연이고, Emoji 없이 가능한 한국어로 답변하세요. 또한, 한자는 한국어로 변환합니다.<|eot_id|>'

In [26]:
human

'<|start_header_id|>user<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'

In [27]:
query

'안녕'

In [31]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    MessagesPlaceholder(variable_name="history"),
    ("human", human),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [32]:
from langchain.agents import AgentExecutor, create_react_agent
agent = create_react_agent(chat,tools,prompt)

ValueError: Prompt missing required variables: {'tool_names', 'tools'}

In [30]:
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    return_intermediate_steps=True
)

NameError: name 'agent' is not defined

In [ ]:
response = agent_executor.invoke(
    {
        "input": query,
        "agent_scratchpad": []
    }
)

In [ ]:
from langchain_core.prompts import PromptTemplate

In [ ]:
template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent
agent = create_react_agent(chat,tools,prompt)

In [ ]:
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    return_intermediate_steps=True
)

In [ ]:
response = agent_executor.invoke({"input": "안녕, 반가워!"})
print(f'답변: {response["output"]}')

In [ ]:
print(response)

In [ ]:
# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [ ]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
)

In [ ]:
def run(agent_executor: AgentExecutor, keyword: str) -> dict:
    result = agent_executor.invoke(
        {
            "input": f"""
        1. `{keyword}` 키워드로 상품 리스트를 가져와줘.
        2. 1번 과정에서 가져온 상품 리스트 각각의 table of content 를 가져와줘.
        3. 마지막으로 모든 상품 리스트의 table of content 를 종합하여 내가 앞으로 집필할 베스트셀러 책의 목차를 작성해 줘.
        """
        }
    )
    return result["output"]

In [ ]:
final_answer = run(agent_executor, "Pandas")